# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import sys
import re
import sqlalchemy as sqla
import pandas as pd
import numpy as np
import pickle
import os
from sklearn import datasets, linear_model
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import make_scorer, accuracy_score, f1_score, fbeta_score, classification_report
from sklearn.externals import joblib
from scipy.stats import hmean
from scipy.stats.mstats import gmean

from matplotlib import pyplot as plt

import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

# import os
# os.getcwd()

[nltk_data] Downloading package punkt to /Users/nomopix/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/nomopix/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/nomopix/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
# load data from database
def load_data():
    '''
        input: none
        output: dataset read split in 
            - X: features
            - y: target
        Description:
            Reads the DisasterResponse.db file and splits it into features and target for further processing.
        NOTES: Depending on which is your working directory you may need to adjust the relative path to the file.
    ''' 
    path_1 = 'sqlite:///Disaster-response-pipelines/data/DisasterResponse.db'
    path_2 = 'sqlite:///../data/DisasterResponse.db'
    try: 
        engine = sqla.create_engine(path_1)
    except:
        engine = sqla.create_engine(path_2)
    
    df = pd.read_sql_query("SELECT * FROM clasified_messages", engine)
    
    # features
    feature_columns = df.columns[~df.columns.isin(['id','original','message','genre'])]
    X = df['message'].values
    # target
    y = df[feature_columns].values

    return X, y, feature_columns


### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    '''
        input: A string of text
        output: a cleaned list of words
        
        description: Given a "text", it will:
            - lower case all words
            - replace url's (if there are any) for a place holder word
            - split into words
            - lemmatize each word (remove variations of endings on words)
            - remove spaces
    '''
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = nltk.word_tokenize(text)
    lemmatizer = nltk.WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):
    '''
    starting verb extractor

    class to, given a text, will return the "root" of verbs rather than their
    conjugated form.
    '''

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

def model_pipeline (parameters = 'none'):
    pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),

        ('reg', MultiOutputClassifier(AdaBoostClassifier(n_estimators=10)))
    ])

    if parameters == 'none':
        parameters = {}

    cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs=4, cv=5)

    return cv

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X, y, category_names = load_data()

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [7]:
pipeline = model_pipeline()

In [8]:
import time
start_time = time.time()
pipeline.fit(X_train, y_train)
print("--- %s seconds ---" % (time.time() - start_time))
cv.best_params_

--- 1051.4819059371948 seconds ---


NameError: name 'cv' is not defined

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [51]:
def evaluate_model(model, X_test, y_test, category_names):
    
    preds = model.predict(X_test)
    for cat in range(0, len(category_names)):
        print('Message category:', category_names[cat])
        print(classification_report(y_test[:, cat], preds[:, cat]))

evaluate_model(pipeline, X_test, y_test, category_names)


Message category: related
              precision    recall  f1-score   support

           0       0.51      0.10      0.17      1590
           1       0.76      0.97      0.85      4912
           2       0.00      0.00      0.00        52

   micro avg       0.75      0.75      0.75      6554
   macro avg       0.42      0.36      0.34      6554
weighted avg       0.70      0.75      0.68      6554

Message category: request
              precision    recall  f1-score   support

           0       0.89      0.97      0.93      5417
           1       0.76      0.45      0.57      1137

   micro avg       0.88      0.88      0.88      6554
   macro avg       0.83      0.71      0.75      6554
weighted avg       0.87      0.88      0.87      6554

Message category: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6527
           1       0.00      0.00      0.00        27

   micro avg       1.00      1.00      1.00      655

### 6. Improve your model
Use grid search to find better parameters. 

In [55]:
params1 = {'features__text_pipeline__vect__ngram_range':[(1,2),(2,2)],
           'reg__estimator__n_estimators':[50, 100]
          }
pipeline1 = model_pipeline(params1)

start_time = time.time()
pipeline1.fit(X_train, y_train)
print("--- %s seconds ---" % (time.time() - start_time))

evaluate_model(pipeline1, X_test, y_test, category_names)

--- 150.6802580356598 seconds ---
Message category: related
              precision    recall  f1-score   support

           0       0.51      0.10      0.17      1590
           1       0.76      0.97      0.85      4912
           2       0.00      0.00      0.00        52

   micro avg       0.75      0.75      0.75      6554
   macro avg       0.42      0.36      0.34      6554
weighted avg       0.70      0.75      0.68      6554

Message category: request
              precision    recall  f1-score   support

           0       0.89      0.97      0.93      5417
           1       0.76      0.45      0.57      1137

   micro avg       0.88      0.88      0.88      6554
   macro avg       0.83      0.71      0.75      6554
weighted avg       0.87      0.88      0.87      6554

Message category: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6527
           1       0.00      0.00      0.00        27

   micro avg      

In [56]:
params2 = {
        'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
        'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0),
        'features__text_pipeline__vect__max_features': (None, 5000, 10000),
        'features__text_pipeline__tfidf__use_idf': (True, False),
        'reg__n_estimators': [50, 100, 200],
        'reg__min_samples_split': [2, 3, 4]
    }
pipeline2 = model_pipeline(params2)

start_time = time.time()
pipeline2.fit(X_train, y_train)
print("--- %s seconds ---" % (time.time() - start_time))

evaluate_model(pipeline2, X_test, y_test, category_names)

--- 144.70057106018066 seconds ---
Message category: related
              precision    recall  f1-score   support

           0       0.51      0.10      0.17      1590
           1       0.76      0.97      0.85      4912
           2       0.00      0.00      0.00        52

   micro avg       0.75      0.75      0.75      6554
   macro avg       0.42      0.36      0.34      6554
weighted avg       0.70      0.75      0.68      6554

Message category: request
              precision    recall  f1-score   support

           0       0.89      0.97      0.93      5417
           1       0.76      0.45      0.57      1137

   micro avg       0.88      0.88      0.88      6554
   macro avg       0.83      0.71      0.75      6554
weighted avg       0.87      0.88      0.87      6554

Message category: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6527
           1       0.00      0.00      0.00        27

   micro avg     

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [ ]:
def save_model(model, model_filepath):
    '''
        input: fitted model
        output: filepath and filename where to save the model
        Description:
            Save a model to a pickl file
    '''

    joblib.dump(model, open(model_filepath, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [ ]:
save_model(